In [1]:
import sys
sys.path.append('../')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision import models
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import TrainDataPrepare as td
import HelperFunctions as hf

sys.path.remove('../')

In [5]:
#traning settings
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001
USE_SYNTHETIC_DATA = True
SYNTHETIC_DATA_DIR = ''
LOAD_ALREADY_MODEL = False

In [2]:
#prepare csv
#prepare data for traning and place them into prepared_data folder
td.SelectDataByPercent('../OOC_image_dataset/train', '../prepared_data/train', 100)
td.SelectDataByPercent('../synthetic_data_0.1/train', '../prepared_data/train', 25)

#transform xlsx file to csv 
hf.convert_xlsx_to_csv('../OOC_datasheet.xlsx', '../OOC_datasheet.csv')

#create csv file also for sythetic data 
td.create_synthetic_csv_from_real_and_structure('../OOC_datasheet.csv', '../synthetic_data_0.1/train', '../OOC_datasheet_01.csv')

#join both csv files toghether
hf.concatenate_csv('../OOC_datasheet.csv', '../OOC_datasheet_01.csv', '../OOC_datasheet_joined.csv')
hf.delete_file('../OOC_datasheet.csv')
hf.delete_file('../OOC_datasheet_01.csv')

#transfer val and test datasets to prepared data
hf.copy_directory('../OOC_image_dataset/test/', '../prepared_data')
hf.copy_directory('../OOC_image_dataset/val/', '../prepared_data')

#split the csv files into train, val, test
hf.split_datasets_by_actual_images('../OOC_datasheet_joined.csv', '../prepared_data', '../prepared_data')
hf.delete_file('../OOC_datasheet_joined.csv')



Headers in the CSV: ['imageID', 'cell type', 'seeding density, cells/ml', 'time after seeding, h', 'day', 'Decision 1/2 (good/bad)', 'flow rate']
Deleted: ../OOC_datasheet.csv
Deleted: ../OOC_datasheet_01.csv
Copied 'c:\Users\Roberts\Documents\repos\Bakalaurs\OOC_image_dataset\test' to 'c:\Users\Roberts\Documents\repos\Bakalaurs\prepared_data\test' successfully.
Copied 'c:\Users\Roberts\Documents\repos\Bakalaurs\OOC_image_dataset\val' to 'c:\Users\Roberts\Documents\repos\Bakalaurs\prepared_data\val' successfully.
Collected 4684 image paths.
Filtered dataset contains 4684 records.
Train data saved to ../prepared_data\train_dataset.csv
Val data saved to ../prepared_data\val_dataset.csv
Test data saved to ../prepared_data\test_dataset.csv
Deleted: ../OOC_datasheet_joined.csv


In [3]:



transform = transforms.Compose([
    #transforms.Resize((600, 600)),  # EfficientNet B7 uses a larger input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# Create datasets
train_dataset = td.CellDataset(csv_file = '../prepared_data/train_dataset.csv', img_dir='../prepared_data/train', transform=transform)
val_dataset = td.CellDataset(csv_file = '../prepared_data/val_dataset.csv', img_dir='../prepared_data/val', transform=transform)
test_dataset = td.CellDataset(csv_file = '../prepared_data/test_dataset.csv', img_dir='../prepared_data/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)




In [4]:
model = models.efficientnet_b7(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

c:\Users\Roberts\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Roberts\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B7_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B7_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-c5b4e57e.pth" to C:\Users\Roberts/.cache\torch\hub\checkpoints\efficientnet_b7_lukemelas-c5b4e57e.pth
100.0%


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormAct

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [8]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        correct = 0
        total = 0
        model.eval()

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f"Epoch {epoch + 1}, Train Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}, Validation Accuracy: {correct / total}")

    print("Training Complete")

In [9]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=EPOCHS)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Roberts\\Documents\\repos\\Bakalaurs\\prepared_data\\train\\synthetic_1_220715_11.png'